In [ ]:
from google.colab import files
files.upload()

In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c dogs-vs-cats

100% 812M/812M [00:30<00:00, 30.9MB/s]
100% 812M/812M [00:30<00:00, 28.4MB/s]


In [5]:
!unzip -qq dogs-vs-cats.zip

In [6]:
!unzip -qq train.zip

In [7]:
import os, shutil, pathlib

original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small")

Utility function to copy cat and dog images from index start_index to index end_index to the subdirectory new_base_dir/{subset_name}/cat (and /dog). The "subset_name" will be either "train", "validation", or "test"

In [8]:
def make_subset (subset_name, start_index, end_index):
  for category in ("cat", "dog"):
    dir = new_base_dir / subset_name / category
    os.makedirs(dir)
    fnames = [f"{category}.{i}.jpg"
              for i in range(start_index, end_index)]
    for fname in fnames:
      shutil.copyfile(src = original_dir / fname,
                      dst = dir / fname)

In [9]:
make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500)
make_subset("test", start_index=1500, end_index=2500)

In [10]:
from tensorflow import keras
from tensorflow.keras import layers

In [11]:
from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(180, 180),
    batch_size=32
)
validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation",
    image_size=(180, 180),
    batch_size=32
)
test_dataset = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(180, 180),
    batch_size=32
)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


In [12]:
for data_batch, labels_batch in train_dataset:
  print("data batch shape:", data_batch.shape)
  print("labels batch shape:", labels_batch.shape)
  break

data batch shape: (32, 180, 180, 3)
labels batch shape: (32,)


In [13]:
conv_base = keras.applications.vgg16.VGG16(
    weights = "imagenet",
    include_top = False,
    input_shape = (180, 180, 3)
)

58889256/58889256 [==============================] - 3s 0us/step


In [14]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 180, 180, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 180, 180, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 90, 90, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 90, 90, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 90, 90, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 45, 45, 128)       0     

In [15]:
from tensorflow.python import train
import numpy as np

def get_feature_and_labels(dataset):
  all_features = []
  all_labels = []
  for images, labels in dataset:
    preprocessed_images = keras.applications.vgg16.preprocess_input(images)
    features = conv_base.predict(preprocessed_images)
    all_features.append(features)
    all_labels.append(labels)
  return np.concatenate(all_features), np.concatenate(all_labels)

train_features, train_labels = get_feature_and_labels(train_dataset)
val_features, val_labels = get_feature_and_labels(validation_dataset)
test_features, test_labels = get_feature_and_labels(test_dataset)

1/1 [==============================] - 6s 6s/step


In [16]:
train_features.shape

(2000, 5, 5, 512)

In [17]:
inputs = keras.Input(shape=(5, 5, 512))
x = layers.Flatten()(inputs)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

In [18]:
import tensorflow as tf
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

history = model.fit(
    train_features, train_labels,
    epochs=20,
    validation_data=(val_features, val_labels))

model.save("feature_extraction.keras")

Epoch 1/20
63/63 [==============================] - 4s 54ms/step - loss: 20.8793 - accuracy: 0.9150 - val_loss: 4.9122 - val_accuracy: 0.9670
Epoch 2/20
63/63 [==============================] - 2s 38ms/step - loss: 3.1632 - accuracy: 0.9815 - val_loss: 10.3312 - val_accuracy: 0.9490
Epoch 3/20
63/63 [==============================] - 4s 60ms/step - loss: 2.5454 - accuracy: 0.9805 - val_loss: 11.6757 - val_accuracy: 0.9490
Epoch 4/20
63/63 [==============================] - 3s 55ms/step - loss: 1.5614 - accuracy: 0.9895 - val_loss: 9.1329 - val_accuracy: 0.9610
Epoch 5/20
63/63 [==============================] - 5s 76ms/step - loss: 0.9732 - accuracy: 0.9910 - val_loss: 8.7753 - val_accuracy: 0.9570
Epoch 6/20
63/63 [==============================] - 2s 38ms/step - loss: 0.9266 - accuracy: 0.9920 - val_loss: 4.2060 - val_accuracy: 0.9770
Epoch 7/20
63/63 [==============================] - 2s 39ms/step - loss: 0.3915 - accuracy: 0.9950 - val_loss: 8.8904 - val_accuracy: 0.9580
Epoch 8/20

In [ ]:
import matplotlib.pyplot as plt
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, "bo", label="Training accuracy")
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("Training and Validation Accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.show()

In [ ]:
conv_base = keras.applications.vgg16.VGG16(
    weights = "imagenet",
    include_top = False
)
conv_base.trainable = False

In [ ]:
conv_base.trainable = True
print("This is the number of trainable weights before freezing the conv base:", len(conv_base.trainable_weights))
conv_base.trainable = False
print("This is the number of trainable weights after freezing the conv base:", len(conv_base.trainable_weights))

In [22]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

In [23]:
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
x = layers.Flatten()(inputs)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

model.compile(loss = "binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [26]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="feature_extraction_with_data_augmentation",
        save_best_only=True,
        monitor="val_loss"
    )
]
history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=validation_dataset,
    callbacks=callbacks)

Epoch 1/50
63/63 [==============================] - 34s 524ms/step - loss: 28497.5000 - accuracy: 0.5210 - val_loss: 4528.6450 - val_accuracy: 0.5570
Epoch 2/50
63/63 [==============================] - 32s 505ms/step - loss: 19877.4492 - accuracy: 0.5225 - val_loss: 14525.3408 - val_accuracy: 0.5000
Epoch 3/50
63/63 [==============================] - 32s 498ms/step - loss: 12989.9248 - accuracy: 0.5230 - val_loss: 13266.3477 - val_accuracy: 0.5000
Epoch 4/50
63/63 [==============================] - 30s 474ms/step - loss: 7666.3442 - accuracy: 0.5135 - val_loss: 1198.3607 - val_accuracy: 0.5760
Epoch 5/50
63/63 [==============================] - 32s 502ms/step - loss: 3223.1758 - accuracy: 0.5130 - val_loss: 3006.1606 - val_accuracy: 0.5000
Epoch 6/50
63/63 [==============================] - 33s 521ms/step - loss: 1118.2512 - accuracy: 0.5165 - val_loss: 1173.0258 - val_accuracy: 0.5000
Epoch 7/50
63/63 [==============================] - 30s 474ms/step - loss: 498.1683 - accuracy: 0.517

In [28]:
test_model = keras.models.load_model(
    "feature_extraction_with_data_augmentation")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc: .3f}")

63/63 [==============================] - 7s 91ms/step - loss: 107.4978 - accuracy: 0.5755
Test accuracy:  0.576


In [29]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

In [30]:
conv_base.trainable = True
for layer in conv_base.layers[:-4]:
    layer.trainable = False

In [32]:
model.compile(loss="binary_crossentropy",
              optimizer=keras.optimizers.RMSprop(learning_rate=1e-5),
              metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="fine_tuning",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks)

Epoch 1/30
63/63 [==============================] - 35s 536ms/step - loss: 27.8272 - accuracy: 0.5610 - val_loss: 13.8259 - val_accuracy: 0.5570
Epoch 2/30
63/63 [==============================] - 33s 522ms/step - loss: 20.3436 - accuracy: 0.5715 - val_loss: 9.1938 - val_accuracy: 0.5490
Epoch 3/30
63/63 [==============================] - 33s 513ms/step - loss: 14.9468 - accuracy: 0.5770 - val_loss: 8.9828 - val_accuracy: 0.5550
Epoch 4/30
63/63 [==============================] - 33s 518ms/step - loss: 11.5327 - accuracy: 0.5655 - val_loss: 5.1330 - val_accuracy: 0.5360
Epoch 5/30
63/63 [==============================] - 85s 1s/step - loss: 7.7426 - accuracy: 0.6015 - val_loss: 4.9647 - val_accuracy: 0.5590
Epoch 6/30
63/63 [==============================] - 32s 498ms/step - loss: 5.0041 - accuracy: 0.5830 - val_loss: 2.1282 - val_accuracy: 0.5420
Epoch 7/30
63/63 [==============================] - 33s 523ms/step - loss: 3.1354 - accuracy: 0.5895 - val_loss: 2.0814 - val_accuracy: 0.52

In [34]:
model = keras.models.load_model("fine_tuning")
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

63/63 [==============================] - 6s 85ms/step - loss: 1.2883 - accuracy: 0.5730
Test accuracy: 0.573
